In [1]:
import networkx as nx
import heapq as hq
from copy import deepcopy

# For some reason pylance doesn't like these imports, but they work fine
import KnowledgeGraph as kg  # type: ignore
import ConceptNode as cn  # type: ignore
from MultiDistance import MultiDistance, multimin, sum_pareto_distance  # type: ignore

In [2]:
G0=nx.read_gpickle('Sample DDI networks/net-ddi-bnu-tau.gpickle')
G1=nx.read_gpickle('Sample DDI networks/net-ddi-cat-tau.gpickle')
G2=nx.read_gpickle('Sample DDI networks/net-ddi-indy-tau.gpickle')
Layers = {'BNU':G0,'CAT':G1,'IND':G2}

In [3]:
all_nodes = set([*list(G0.nodes()),*list(G1.nodes()),*list(G2.nodes())])

for node in all_nodes:
    for G in Layers.values():
        if node not in G.nodes():
            G.add_node(node)

set(Layers['BNU'].nodes())==set(Layers['CAT'].nodes())==set(Layers['IND'].nodes())

True

In [4]:
print(f'{len(all_nodes)=}')
print(f'{Layers["BNU"].number_of_edges()=}')
print(f'{Layers["CAT"].number_of_edges()=}')
print(f'{Layers["IND"].number_of_edges()=}')
print(f'total edges={Layers["BNU"].number_of_edges()+Layers["CAT"].number_of_edges()+Layers["IND"].number_of_edges()}')


len(all_nodes)=381
Layers["BNU"].number_of_edges()=146
Layers["CAT"].number_of_edges()=708
Layers["IND"].number_of_edges()=776
total edges=1630


In [5]:
KG = kg.KnowledgeGraph(Layers)

In [6]:
from multiprocessing import Pool
import mp_for_nb


arglist = mp_for_nb.make_arglist(KG,'BNU')

with Pool() as pool:
    results = pool.imap_unordered(mp_for_nb.appd_cut,arglist)
    APPD = list(results)
    
closure = {k[0]: {vk[0]: vv for vk, vv in v.items()} for k, v in APPD}

removed_edges_mp = KG.pareto_backbone_removed_edges(closure=closure)

In [7]:
removed_edges = KG.pareto_backbone_removed_edges(depth_cut=2)

In [8]:
print(removed_edges_mp)
print("----")
print(removed_edges)

{(('DB01026', 'CAT'), ('DB00635', 'CAT')), (('DB01167', 'CAT'), ('DB00635', 'CAT')), (('DB01367', 'CAT'), ('DB01242', 'CAT')), (('DB01045', 'CAT'), ('DB01234', 'CAT')), (('DB01367', 'CAT'), ('DB00715', 'CAT')), (('DB01060', 'CAT'), ('DB00254', 'CAT')), (('DB00227', 'CAT'), ('DB00682', 'CAT')), (('DB01242', 'CAT'), ('DB01367', 'CAT')), (('DB01076', 'CAT'), ('DB00564', 'CAT')), (('DB01174', 'BNU'), ('DB00682', 'BNU')), (('DB00864', 'CAT'), ('DB01167', 'CAT')), (('DB01104', 'CAT'), ('DB00252', 'CAT')), (('DB00953', 'CAT'), ('DB00320', 'CAT')), (('DB00252', 'CAT'), ('DB00404', 'CAT')), (('DB00682', 'CAT'), ('DB00564', 'CAT')), (('DB01394', 'CAT'), ('DB01211', 'CAT')), (('DB00252', 'CAT'), ('DB01104', 'CAT')), (('DB00641', 'CAT'), ('DB00091', 'CAT')), (('DB00682', 'BNU'), ('DB01174', 'BNU')), (('DB00564', 'CAT'), ('DB01418', 'CAT')), (('DB01367', 'CAT'), ('DB00476', 'CAT')), (('DB01167', 'CAT'), ('DB00959', 'CAT')), (('DB00959', 'CAT'), ('DB01167', 'CAT')), (('DB01167', 'CAT'), ('DB01211', 

In [9]:
import distanceclosure as dc

In [10]:
print(G0.number_of_edges())
print(len(G0.edges()))
print(nx.DiGraph(G0).number_of_edges())

146
146
292


In [13]:
BB0=dc.distance_closure(G0,only_backbone=True)
BB1=dc.distance_closure(G1,only_backbone=True)
BB2=dc.distance_closure(G2,only_backbone=True)

removed_edges_BB = []
for BB,L in zip( [BB0,BB1,BB2], ['BNU','CAT','IND']):
    for u,v,d in list(BB.edges(data=True)):
        if not d['is_trig']:
            removed_edges_BB.append(((u,L),(v,L)))
            BB.remove_edge(u,v)


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (distance.py, line 135)

In [ ]:
sre = set(removed_edges | set((v,u) for u,v in removed_edges))
sreBB = set(removed_edges_BB+[(v,u) for u,v in removed_edges_BB])
print(sreBB-sre)
print(sre-sreBB)
print(len(sreBB)/2)
print(len(sre)/2)

{(('DB00857', 'CAT'), ('DB00514', 'CAT')), (('DB00514', 'CAT'), ('DB00857', 'CAT'))}
set()
53.0
52.0


In [ ]:
LayersBB = {'BNU':BB0,'CAT':BB1,'IND':BB2}

In [ ]:
all_nodesBB = set([*list(BB0.nodes()),*list(BB1.nodes()),*list(BB2.nodes())])

In [ ]:
for node in all_nodesBB:
    for G in LayersBB.values():
        if node not in G.nodes():
            G.add_node(node)

set(LayersBB['BNU'].nodes())==set(LayersBB['CAT'].nodes())==set(LayersBB['IND'].nodes())

True

In [ ]:
print(f'{len(all_nodesBB)=}')
print(f'{len(LayersBB["BNU"].edges())=}')
print(f'{len(LayersBB["CAT"].edges())=}')
print(f'{len(LayersBB["IND"].edges())=}')
print(f'{len(LayersBB["BNU"].edges())+len(LayersBB["CAT"].edges())+len(LayersBB["IND"].edges())=}')


len(all_nodesBB)=381
len(LayersBB["BNU"].edges())=143
len(LayersBB["CAT"].edges())=660
len(LayersBB["IND"].edges())=774
len(LayersBB["BNU"].edges())+len(LayersBB["CAT"].edges())+len(LayersBB["IND"].edges())=1577


In [ ]:
KGBB = kg.KnowledgeGraph(LayersBB)

In [ ]:
cBB=KGBB.pareto_distance_closure(depth_cut=3,start_layer="IND")

In [ ]:
KGBB.pareto_backbone_removed_edges(closure=cBB)

set()